In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
url = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001'
yes24 = requests.get(url)

In [ ]:
bs = BeautifulSoup(yes24.text)
titlelist = []
authorlist = []
publisherlist = []
pricelist = []

In [ ]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    title = bs.find('ol', class_ = '') \
            .find('li', class_ = 'num' + idx) \
            .find_all('p')[2].text
    title = title.replace('[도서] ', '')
    titlelist.append(title.strip())

In [ ]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    author = bs.find('ol', class_ = '') \
                .find('li', class_ = 'num' + idx) \
                .find('p', class_ = 'aupu').text
    author = author.split('|')[0].strip()
    authorlist.append(author)

In [ ]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    price = bs.find('ol', class_ = '') \
            .find('li', class_ = 'num' + idx) \
            .find('p', class_ = 'price').text
    price = price.split('원')[0]
    price = int(re.sub(r',', '', price))
    pricelist.append(price)

In [ ]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    publisher = bs.find('ol', class_ = '') \
            .find('li', class_ = 'num' + idx) \
            .find('p', class_ = 'aupu').text
    publisher = publisher.split('|')[1]
    publisherlist.append(publisher)

In [ ]:
import pandas as pd

df = pd.DataFrame(titlelist, columns = ['Title'])

In [ ]:
df['Author'] = authorlist
df['Publisher'] = publisherlist
df['Price'] = pricelist

In [ ]:
import numpy as np
df['Price'].astype('int64')

In [ ]:
df.info()

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('yes24.db')
cursor = conn.cursor()

sql = 'DROP TABLE IF EXISTS BestSeller'
cursor.execute(sql)
sql = '''CREATE TABLE BestSeller(
         idx      INT    PRIMARY KEY,
         title    TEXT,
         author   TEXT,
         publisher  TEXT,
         price     INT
)'''
cursor.execute(sql)

In [ ]:
for i in range(0, len(df)) : 
    cursor.execute("INSERT INTO BestSeller VALUES(" + str(i) + ", \'" + df.loc[i, 'Title'] + "\',\'" + df.loc[i, 'Author'] + "\',\'" + df.loc[i, 'Publisher'] + "\', " + str(df.loc[i, 'Price']) + ")")
    conn.commit()

In [ ]:
cursor.close()
conn.close()